In [ ]:
import holoviews as hv, holoviews.streams as hvs
import geoviews as gv

hv.extension('bokeh')

## Annotating ML data in Python with Bokeh and HoloViews

The [Bokeh](https://bokeh.org) Python plotting library lets users build interactive apps and plots in a web browser for a very wide variety of data types.  The high-level library [HoloViews](https://holoviews.org) builds on Bokeh, making it easier to use for common data-processing tasks, and the corresponding [GeoViews](https://geoviews.org) library adds support for plotting in geographic coordinate systems. 

These tools now (as of development releases in 1/2020) all support interactively collecting data _from_ the user, not just interacting with existing data, with components provided by HoloViews (and by GeoViews for data on maps) that make it simple to get data into Python ready to process and use for tagging data for machine-learning pipelines (or any other purpose!). These "drawing" and "annotation" tools can be used to annotate existing data sets or geographic locations, to classify each example or regions into categories or with numeric or other labels.

These tools make it possible to work directly with data in its native values (*as data*) and then immediately use it for further processing in Python. Other tools like [labelImg](https://github.com/tzutalin/labelImg) will usually be faster and easier to use for the specific things they do, so if one of those meets your need, use it! Meanwhile, use the Bokeh/HoloViews annotation tools if you want to quickly create a fully custom app for special purposes, especially if you want to stay working with data you are already using in Python, in its native coordinates.

Here, we will focus primarily on the easy-to-use, high-level ["annotator" components from HoloViews](http://build.holoviews.org/user_guide/Annotators.html); fully custom control is always available by using [Bokeh's drawing tools](https://docs.bokeh.org/en/latest/docs/reference/models/tools.html#bokeh.models.tools.PointDrawTool) directly.

### Basic HoloViews/GeoViews usage

Let's imagine our task is to mark the locations of trees on satellite images. For convenience, we'll use a tile-based web mapping server where these images have been cleaned up and put into geographic lat,lon coordinates already. In GeoViews, you can easily get a tile source to work with:

In [ ]:
tiles = gv.tile_sources.EsriImagery()
tiles

Next, we'll need an object to collect some lat,lon locations. Here's an example with three points already identified: 

In [ ]:
pts = dict(
    Longitude = [-121.932619100, -121.932362392, -121.933530027],
    Latitude  = [  36.631164244,   36.629475356,   36.630623206])
    
opts = dict(size=10, line_color='black', padding=0.1, min_height=400)

points = gv.Points(pts).opts(**opts)
points

These particular points are locations of actual trees somewhere in Monterey, California, as you can see if you overlay them onto the tiles (where `*` means "overlay" in HoloViews):

In [ ]:
tiles * points

## HoloViews PointDraw tool

The Bokeh tools in the tool bar above let you pan and zoom on that plot interactively, but the data in it is fixed. What if we wanted to label all the trees that we can see there, i.e. add more data points? Bokeh and HoloViews support "drawing tools" that can edit data on plots, and it's easy to add a drawing tool to add more points wherever you like:

In [ ]:
point_stream = hvs.PointDraw(data=points.columns(), source=points, empty_value=None)
points.opts(active_tools=['point_draw'], tools=['hover'])
tiles * points

You'll see that there is now a [PointDraw tool](../reference/streams/bokeh/PointDraw.ipynb) in the toolbar:<img src="https://bokeh.pydata.org/en/latest/_images/PointDraw.png">

Once you select that tool, you should be able to click and drag any of the existing points to a new location, as well as click to add a new point anywhere there is not already one. You can delete points by selecting them in the plot and pressing Backspace or Delete (depending on operating system).

Try clicking on some of the trees that haven't been labeled. Once you are happy, you can get the data back into Python to do whatever you want with it, whether that be saving to a file or immediately performing some computations:

In [ ]:
import pandas as pd
pd.DataFrame(point_stream.data)

The other drawing tools in HoloViews work much the same way, allowing you to add [boxes](http://holoviews.org/reference/streams/bokeh/BoxEdit.html#bokeh-gallery-boxedit),
[freehand curves](http://holoviews.org/reference/streams/bokeh/FreehandDraw.html#bokeh-gallery-freehanddraw),
or [polygons](http://holoviews.org/reference/streams/bokeh/PolyDraw.html#bokeh-gallery-polydraw) ([edited separately](http://holoviews.org/reference/streams/bokeh/PolyEdit.html#bokeh-gallery-polyedit)).

## HoloViews Point Annotator

What if you also want to associate some numeric, textual, or category label or value with each new data item or datapoint you add? You can do that manually using the drawing tools by adding a separate table and linking it to your element (e.g. Points). To make that easy, HoloViews provides "Annotator" objects that package the element, drawing tool, and editable table together into a single object.

A HoloViews (or GeoViews) annotator lets you add, change, or add information to data in a Bokeh plot, then get the data back into Python easily. Here, let's make an annotator for the points, then overlay the annotated points on the map tiles like we had before:

In [ ]:
points_annotator = hv.annotate.instance()
points_annotator(tiles * points, annotations=dict(Size=int, Type=str))

You'll see that there is now a table of coordinates along side your plot, and that you still have a drawing tool in the toolbar. As you click on an existing point, you should be able to click and drag to see the location update in the table. Whether you click on the table or the points, the same object should be selected in each, so that you can see how the graphical and tabular representations relate.

When you add new points, a new table row should appear ready for editing. For deleting points, you can select them either on the plot or as a row in the table, but the mouse pointer needs to be in the plot area for Backspace/Delete to take effect.

Whether for existing or newly added points, you can use the table to edit the latitude and longitude values numerically or add an optional "Size" estimate or "Type" description for each point.

There are also save and restore tools that help make sure you don't lose work once you've added a lot of data, but we won't have time to cover those here.

Now that we've added some points, let's get the data back into Python as a Pandas DataFrame:

In [ ]:
points_annotator.annotated.dframe()

You should see that you can access the current set of user-provided or user-modified points and their user-provided labels from within Python, ready for saving to disk or any subsequent processing you need to do.

We can also access the currently `selected` points, in case we care only about a subset of the points (which will be empty if no points/rows are selected):

In [ ]:
points_annotator.selected.dframe()

## HoloViews Rectangle Annotator

HoloViews data types that can currently be annotated include:

* ``Points``/``Scatter``
* ``Curve``
* ``Path``
* ``Polygons``
* ``Rectangles``

Let's look at the `Rectangles` annotator, which behaves very similarly to the Points annotator:

* Select the `BoxEdit` tool in the toolbar: <img src="https://bokeh.pydata.org/en/latest/_images/BoxEdit.png">
* Click and drag on an existing Rectangle to move it
* Double click to start drawing a new Rectangle at one corner, and double click to complete the rectangle at the opposite corner
* Select a rectangle and press the Backspace or Delete key (depending on OS) to delete it, while pointing at the plot (not the table)
* Edit the box coordinates in the table to resize it

In [ ]:
rectangles = gv.Rectangles([(0, 0, 3, 3), (12, 12, 15, 15)]).opts(fill_alpha=0.2)
box_annotator = hv.annotate.instance()
labels = gv.tile_sources.StamenLabels()

box_annotator(tiles * rectangles * labels, name="Rectangles")

As for Points, we can access the data using the ``annotated`` property on the annotator instance, and then use these coordinates as part of any subsequent workflow:

In [ ]:
box_annotator.annotated.dframe()

### Annotating paths/polygons

Annotated points associate data with each point, and annotated Rectangles associate data with the entire Rectangle. Annotated Paths and Polygons allow both, i.e. associating one value with the entire object ("this polygon is Arizona"), and associating specific values with each vertex of the object ("this position along the border has elevation X"). This capability makes these annotators more complex (see the [HoloViews Annotators user guide](http://build.holoviews.org/user_guide/Annotators.html) and the [PolyDraw](../reference/streams/PolyDraw.ipynb) and [PolyEdit](../reference/streams/PolyEdit.ipynb) docs for more details), but we'll do a brief demo here.

##### Drawing/Selecting Deleting Paths/Polygons

- Select the PolyDraw tool in the toolbar: <img src="https://bokeh.pydata.org/en/latest/_images/PolyDraw.png">
- Double click to start a new object, single click to add each vertex, and double-click to complete it.
- Delete paths/polygons by selecting and pressing Delete key (OSX) or Backspace key (PC)

##### Editing Paths/Polygons

- Select the PolyEdit tool in the toolbar: <img src="https://bokeh.pydata.org/en/latest/_images/PolyEdit.png">
- Double click a Path/Polygon to start editing
- Drag vertices to edit them, delete vertices by selecting them

To edit and annotate the vertices, use the draw tool or the first table to select a particular path/polygon and then navigate to the Vertices tab.

In [ ]:
path = gv.Path([([-3.208222, -3.203861, -3.203865, -3.202945, -3.205764, -3.208222], 
                 [55.868081, 55.867272, 55.867866, 55.868922, 55.869360, 55.868081]),
                ([-3.208646, -3.206124, -3.208234, -3.211137, -3.208646], 
                 [55.864370, 55.863135, 55.861888, 55.862793, 55.864370])])

path_annotator = hv.annotate.instance()
path_annotator(tiles * path, annotations=['Label'], vertex_annotations=['Value'])

To access the data we can make use of the iloc method on `Path` objects to access a particular path, and then access the `.data` or convert it to a dataframe:

In [ ]:
path_annotator.annotated.iloc[0].dframe()

By the way, for any of the annotators but most usefully for paths and polygons, we can also get the data back as a [Shapely](https://shapely.readthedocs.io) geometry if that's more convenient to work with:

In [ ]:
path_annotator.annotated.geom()

In [ ]:
path_annotator.annotated.iloc[0].geom()

# Integrating Annotators into your workflows

As you can see above, it's fairly straightforward to use a drawing tool or build an annotator to collect a specific type of data. To collect data at a large scale, you'll want to focus on usability, which will often mean creating a special-purpose app to collect data across multiple images, multiple datasets, by multiple raters, etc. Doing so is beyond the scope of this introduction, but can be straightforward using the separate [Panel](https://panel.holoviz.org) library for building apps, which is also based on Bokeh and has full support for HoloViews. Annotator objects can be included directly in a Panel layout and connected to other Panel objects for seamless updating and integration into a larger workflow.

For more details, see:

- [HoloViews annotator docs](http://build.holoviews.org/user_guide/Annotators.html)
- [HoloViz.org](https://holoviz.org) for a full tutorial on all the various libraries in the HoloViz ecosystem (including HoloViews, GeoViews, and Panel).
- [EarthML.pyviz.org](https://EarthML.pyviz.org) for Earth-science related ML examples
- [EarthSim.pyviz.org](https://EarthSim.pyviz.org) for hydrology-related simulation examples

And please [let us know](https://github.com/holoviz/holoviews) if you find any rough edges or missing functionality in the annotators; these are relatively new to Bokeh, HoloViews, and GeoViews, and can probably be improved as more people try them out for new tasks as long as we know what's needed!